Combinar DataFrames con concat()

En esta lección volveremos al conjunto de datos de ventas de videojuegos. Aquí están las primeras filas para recordarte su estructura:

In [ ]:
import pandas as pd

df = pd.read_csv('/datasets/vg_sales.csv')
print(df.head())

"""                       name platform  year_of_release         genre publisher  \
0                Wii Sports      Wii           2006.0        Sports  Nintendo   
1         Super Mario Bros.      NES           1985.0      Platform  Nintendo   
2            Mario Kart Wii      Wii           2008.0        Racing  Nintendo   
3         Wii Sports Resort      Wii           2009.0        Sports  Nintendo   
4  Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing  Nintendo   

  developer  na_sales  eu_sales  jp_sales  critic_score  user_score  
0  Nintendo     41.36     28.96      3.77          76.0         8.0  
1       NaN     29.08      3.58      6.81           NaN         NaN  
2  Nintendo     15.68     12.76      3.79          82.0         8.3  
3  Nintendo     15.61     10.93      3.28          80.0         8.0  
4       NaN     11.27      8.89     10.22           NaN         NaN"""

Queremos conocer algunas estadísticas generales de las distribuidoras de juegos, en particular:

- su puntuación promedio de la crítica;
- sus ventas totales.

Como ya hemos visto, podemos hacerlo usando groupby(). Primero, obtengamos la puntuación de reseña promedio para cada distribuidora:

In [ ]:
import pandas as pd

df = pd.read_csv('/datasets/vg_sales.csv')

mean_score = df.groupby('publisher')['critic_score'].mean()
print(mean_score)

"""
publisher
10TACLE Studios                 42.000000
1C Company                      73.000000
20th Century Fox Video Games          NaN
2D Boy                          90.000000
3DO                             57.470588
                                  ...    
id Software                     85.000000
imageepoch Inc.                       NaN
inXile Entertainment            81.000000
mixi, Inc                             NaN
responDESIGN                          NaN
Name: critic_score, Length: 581, dtype: float64"""

Obtengamos también el número de ventas. La forma más sencilla de hacerlo es con un segundo groupby():

In [ ]:
df['total_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales']
num_sales = df.groupby('publisher')['total_sales'].sum()
print(num_sales)

"""
publisher
10TACLE Studios                 0.11
1C Company                      0.08
20th Century Fox Video Games    1.92
2D Boy                          0.03
3DO                             9.52
                                ... 
id Software                     0.02
imageepoch Inc.                 0.04
inXile Entertainment            0.09
mixi, Inc                       0.87
responDESIGN                    0.13
Name: total_sales, Length: 581, dtype: float64"""

Observa que el índice de ambos resultados es la columna 'publisher' porque agrupamos por 'publisher' en ambos casos. Como ambos resultados comparten el mismo índice, podemos unir fácilmente los resultados en un DataFrame usando la función concat() de pandas:

In [ ]:
import pandas as pd

df = pd.read_csv('/datasets/vg_sales.csv')

mean_score = df.groupby('publisher')['critic_score'].mean()

df['total_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales']
num_sales = df.groupby('publisher')['total_sales'].sum()

df_concat = pd.concat([mean_score, num_sales], axis='columns')
print(df_concat)

"""                              critic_score  total_sales
publisher                                              
10TACLE Studios                  42.000000         0.11
1C Company                       73.000000         0.08
20th Century Fox Video Games           NaN         1.92
2D Boy                           90.000000         0.03
3DO                              57.470588         9.52
...                                    ...          ...
id Software                      85.000000         0.02
imageepoch Inc.                        NaN         0.04
inXile Entertainment             81.000000         0.09
mixi, Inc                              NaN         0.87
responDESIGN                           NaN         0.13

[581 rows x 2 columns]"""

En general, concat() espera una lista de objetos de tipo Series y/o DataFrame. Para obtener nuestro resultado, pasamos una lista de variables de Series a concat() y configuramos axis='columns' para asegurarnos de que se combinaran como columnas.

Ten en cuenta que los nombres de las columnas originales se conservan en el DataFrame concatenado.

Podemos renombrar columnas utilizando el método columns. Se puede llamar en un DataFrame y pasarle una lista de nuevos nombres de columnas para reemplazar las existentes. Los nuevos nombres deben pasarse en el mismo orden que los nombres originales de las columnas.

Cambiemos el nombre de 'critic_score', ya que ahora representa un promedio:

In [ ]:
import pandas as pd

df = pd.read_csv('/datasets/vg_sales.csv')

mean_score = df.groupby('publisher')['critic_score'].mean()

df['total_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales']
num_sales = df.groupby('publisher')['total_sales'].sum()

df_concat = pd.concat([mean_score, num_sales], axis='columns')
#change original column names for new column names
df_concat.columns = ['avg_critic_score', 'total_sales']
print(df_concat)

"""                              avg_critic_score  total_sales
publisher                                                  
10TACLE Studios                      42.000000         0.11
1C Company                           73.000000         0.08
20th Century Fox Video Games               NaN         1.92
2D Boy                               90.000000         0.03
3DO                                  57.470588         9.52
...                                        ...          ...
id Software                          85.000000         0.02
imageepoch Inc.                            NaN         0.04
inXile Entertainment                 81.000000         0.09
mixi, Inc                                  NaN         0.87
responDESIGN                               NaN         0.13

[581 rows x 2 columns]"""

En general, es una buena idea cambiar el nombre de las columnas después de agruparlas y procesarlas para que sean más indicativas de cómo se procesaron las columnas.

Es posible que hayas notado que podríamos obtener el mismo resultado que antes usando agg(). Sin embargo, concat() es bastante versátil. Podemos utilizarlo para concatenar DataFrames:

- por filas, suponiendo que tengan el mismo número de columnas;
- por columnas si tienen el mismo número de filas.

Para concatenar filas de DataFrames separados, podemos usar concat() y configurar axis='index' (u omitir este parámetro, ya que axis='index' es el argumento predeterminado). Alternativamente, podemos utilizar números enteros para el argumento index=, donde index=0 concatenará filas y index=1 concatenará columnas.

Aquí hay un ejemplo en el que filtramos los datos en dos DataFrames separados según el género y luego los recombinamos en un solo DataFrame:

In [ ]:
import pandas as pd

df = pd.read_csv('/datasets/vg_sales.csv')

rpgs = df[df['genre'] == 'Role-Playing']
platformers = df[df['genre'] == 'Platform']

df_concat = pd.concat([rpgs, platformers])
print(df_concat[['name', 'genre']])

"""                                                name         genre
4                              Pokemon Red/Pokemon Blue  Role-Playing
12                          Pokemon Gold/Pokemon Silver  Role-Playing
20                        Pokemon Diamond/Pokemon Pearl  Role-Playing
25                        Pokemon Ruby/Pokemon Sapphire  Role-Playing
27                          Pokemon Black/Pokemon White  Role-Playing
...                                                 ...           ...
16356                                    Strider (2014)      Platform
16358                                Goku Makaimura Kai      Platform
16603  The Land Before Time: Into the Mysterious Beyond      Platform
16710                Woody Woodpecker in Crazy Castle 5      Platform
16715                                  Spirits & Spells      Platform"""

¡Y así dos DataFrames se unen en uno! Recuerda que esto funciona aquí porque ambos DataFrames más pequeños tienen las mismas columnas.

EJERCICIOS

1.

Leímos los datos, creamos una columna 'total_sales' y calculamos las ventas totales para cada plataforma en la variable total_sales.

Tienes que calcular el número total de distribuidoras que crearon un juego en cada plataforma, utilizando nunique(). Asigna el resultado a una variable llamada num_pubs y luego muéstralo.

In [ ]:
import pandas as pd

df = pd.read_csv('/datasets/vg_sales.csv')
df['total_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales']

total_sales = df.groupby('platform')['total_sales'].sum()

num_pubs = df.groupby('platform')['publisher'].nunique() # escribe tu código aquí

print(num_pubs)# imprime aquí

"""Llama a groupby() en df para agrupar por 'platform', luego llama 
a nunique() en la columna 'publisher' del DataFrame agrupado. 
Asigna el resultado a una variable llamada num_pubs y luego muéstralo."""

"""platform
2600     26
3DO       3
3DS      82
DC       15
DS      175
GB       17
GBA      87
GC       52
GEN       7
GG        1
N64      54
NES      12
NG        3
PC      133
PCFX      1
PS      151
PS2     172
PS3     103
PS4      75
PSP     127
PSV      66
SAT      44
SCD       1
SNES     50
TG16      2
WS        2
Wii     113
WiiU     23
X360    102
XB       73
XOne     46
Name: publisher, dtype: int64"""

2.

Combina total_sales y num_pubs por columnas en un DataFrame llamado platforms usando concat(). Cambia los nombres de las columnas en platforms a 'total_sales' y 'num_publishers', respectivamente, luego imprime platforms.

In [ ]:
import pandas as pd

df = pd.read_csv('/datasets/vg_sales.csv')
df['total_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales']

total_sales = df.groupby('platform')['total_sales'].sum()
num_pubs = df.groupby('platform')['publisher'].nunique()

platforms = pd.concat([total_sales, num_pubs], axis='columns') # escribe tu código aquí

platforms.columns = ['total_sales', 'num_publishers']# cambia los nombres de las columnas aquí

print(platforms)# muestra tu resultado


"""         total_sales  num_publishers
platform
2600            96.07              26
3DO              0.10               3
3DS            245.64              82
DC              15.68              15
DS             747.13             175
GB             247.26              17
GBA            310.12              87
GC             193.75              52
GEN             27.46               7
GG               0.04               1
N64            214.30              54
NES            245.74              12
NG               1.44               3
PC             237.14             133
PCFX             0.03               1
PS             689.95             151
PS2           1062.33             172
PS3            803.97             103
PS4            265.83              75
PSP            252.63             127
PSV             47.63              66
SAT             33.52              44
SCD              1.81               1
SNES           196.82              50
TG16             0.16               2
WS               1.42               2
Wii            828.44             113
WiiU            76.24              23
X360           885.66             102
XB             249.02              73
XOne           145.05              46"""